In [2]:
import numpy as np
import os 
import tensorflow as tf 
import tensorflow_datasets as tfds 
keras = tf.keras
import cv2
import json


2022-08-17 13:22:15.484088: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

AffectNet_train = ALL_DATA_DIR + 'AffectNet_train_set/'
AffectNet_train_annotation = AffectNet_train + 'annotations/'
AffectNet_train_images = AffectNet_train + 'images/'


AffectNet_val = ALL_DATA_DIR + 'AffectNet_val_set/'
AffectNet_val_annotation = AffectNet_val + 'annotations/'
AffectNet_val_images = AffectNet_val + 'images/'

#scaled_anger = scaled + 'scaled_disgust/one/''
print(training)
print(validation)
print(scaled)
print(scaled_wo_rotate)
print(AffectNet_val_images)

/media/windows/training/
/media/windows/validation/
/media/windows/scaled/
/media/windows/scaled_wo_rotate/
/media/windows/AffectNet_val_set/images/


In [4]:
data1 = np.load(AffectNet_train_annotation + '2_aro.npy')
data2 = np.load(AffectNet_train_annotation + '2_exp.npy')
data3 = np.load(AffectNet_train_annotation + '2_lnd.npy')
data4 = np.load(AffectNet_train_annotation + '2_val.npy')

print(data1)
print(data2)
print(data3)
print(data4)


0.00793651
0
[ 24.56449438  91.43272727  27.11910112 114.31187166  32.64359551
 137.52641711  40.34516854 159.2915508   51.29348315 178.10994652
  65.48853933 194.35294118  82.32629213 206.60705882  99.5541573
 216.38160428 120.49438202 218.72941176 141.63595506 214.5368984
 162.84044944 205.64877005 182.38382022 191.5259893  198.98247191
 174.43251337 210.4341573  153.13454545 215.83280899 129.33304813
 218.84044944 106.50181818 218.07280899  80.8915508   32.42966292
  67.47550802  43.28988764  58.39572193  58.41617978  55.16149733
  74.61213483  55.98802139  89.93977528  61.71379679 123.31325843
  59.29411765 142.12674157  52.31058824 161.35550562  49.36385027
 180.40808989  52.29860963 196.80539326  60.30032086 108.89168539
  85.41946524 109.30696629  99.32663102 109.65932584 113.22181818
 110.3011236  127.4284492   95.08674157 143.36       103.39235955
 144.29433155 111.86157303 144.9771123  121.01033708 143.06053476
 129.90741573 141.31165775  47.1658427   89.76770053  58.26516854

In [5]:
batch_size = 32 
img_height = 224
img_width = 224 

### with 7 emotions (added neutral into the dataset)

In [6]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    scaled_wo_rotate_validation, 
    labels = 'inferred', 
    image_size=(img_height,img_width)
    )

Found 50947 files belonging to 7 classes.


2022-08-17 13:22:39.814129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:22:39.898303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:22:39.901302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-17 13:22:39.906717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

### create the program that distributes the file in the 'images' file
#### first create the json 

In [165]:
os.chdir(AffectNet_val_annotation)
annotations = os.listdir()
label_list = []
for i in annotations:
    #print(i[-7:])
    if i[-7:] == "exp.npy":
        label_list.append(i)

print(len(label_list))

#label_list is the list with the exp.npy name 
#create json with filename and lable. 
#file name is number.jpg lable is int
made_metadata = []
#for i in range(100):
#    print(label_list[i][:-8]) #filename
for i in label_list:
    dic = {}
    dic['filename'] = i[:-8]+".jpg"
    dic['label'] = int(np.load(i))
    made_metadata.append(dic)
print(len(made_metadata))
path = AffectNet_val + "AFN_val.json"
with open(path ,'w',encoding='utf-8')as make_file:
    json.dump(made_metadata, make_file, indent = "\t")



3999
3999


### distritute

In [183]:
with open(AffectNet_val+ "AFN_val.json", 'r', encoding = "utf-8")as f:
    data = json.load(f)
for i in range(len(data)):
    image = cv2.imread(AffectNet_val_images+data[i]["filename"])
    #print(image)
    path = AffectNet_val + "distributed/"+ str(data[i]["label"])+'/'
    #print(path)
    cv2.imwrite(path+data[i]["filename"], image)

In [184]:
AfNet_val_ds = tf.keras.utils.image_dataset_from_directory(
    AffectNet_val+"distributed/",
    labels = "inferred",
    image_size = (224,224)
    )

Found 3999 files belonging to 8 classes.


In [185]:
os.chdir(AffectNet_train_annotation)
annotations = os.listdir()
label_list = []
for i in annotations:
    #print(i[-7:])
    if i[-7:] == "exp.npy":
        label_list.append(i)

print(len(label_list))

#label_list is the list with the exp.npy name 
#create json with filename and lable. 
#file name is number.jpg lable is int
made_metadata = []
#for i in range(100):
#    print(label_list[i][:-8]) #filename
for i in label_list:
    dic = {}
    dic['filename'] = i[:-8]+".jpg"
    dic['label'] = int(np.load(i))
    made_metadata.append(dic)
print(len(made_metadata))
path = AffectNet_train + "AFN_train.json"
with open(path ,'w',encoding='utf-8')as make_file:
    json.dump(made_metadata, make_file, indent = "\t")


287651
287651


In [187]:
with open(AffectNet_train+ "AFN_train.json", 'r', encoding = "utf-8")as f:
    data = json.load(f)
k = 0
counter = 0
for i in range(len(data)):
    image = cv2.imread(AffectNet_train_images+data[i]["filename"])
    #print(image)
    path = AffectNet_train + "distributed/"+ str(data[i]["label"])+'/'
    #print(path)
    cv2.imwrite(path+data[i]["filename"], image)
    k = k+1
    if k == 10000:
        k = 0
        counter = counter + 1
        print(counter)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [189]:
AfNet_train_ds = tf.keras.utils.image_dataset_from_directory(
    AffectNet_train+"distributed/",
    labels = "inferred",
    image_size = (224,224)
    )

Found 287651 files belonging to 8 classes.


In [7]:
training_ds = tf.keras.utils.image_dataset_from_directory(
    scaled_wo_rotate_training, 
    labels = 'inferred', 
    image_size=(img_height,img_width)
    )

Found 399839 files belonging to 7 classes.


### base model with weight 'imagenet'

In [8]:
base_model = tf.keras.applications.MobileNetV3Small(input_shape = (224,224,3), include_top = False, weights='imagenet')
base_model.trainable = False 
base_model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   

 expanded_conv_1/depthwise/Batc  (None, 28, 28, 72)  288         ['expanded_conv_1/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 re_lu_4 (ReLU)                 (None, 28, 28, 72)   0           ['expanded_conv_1/depthwise/Batch
                                                                 Norm[0][0]']                     
                                                                                                  
 expanded_conv_1/project (Conv2  (None, 28, 28, 24)  1728        ['re_lu_4[0][0]']                
 D)                                                                                               
                                                                                                  
 expanded_conv_1/project/BatchN  (None, 28, 28, 24)  96          ['expanded_conv_1/project[0][0]']
 orm (Batc

                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    2328        ['expanded_conv_3/squeeze_excite/
 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    0           ['expanded_conv_3/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 96)    2400        ['expanded_conv_3/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__oper

                                                                                                  
 expanded_conv_4/squeeze_excite  (None, 14, 14, 240)  0          ['multiply_4[0][0]',             
 /Mul (Multiply)                                                  'tf.math.multiply_7[0][0]']     
                                                                                                  
 expanded_conv_4/project (Conv2  (None, 14, 14, 40)  9600        ['expanded_conv_4/squeeze_excite/
 D)                                                              Mul[0][0]']                      
                                                                                                  
 expanded_conv_4/project/BatchN  (None, 14, 14, 40)  160         ['expanded_conv_4/project[0][0]']
 orm (BatchNormalization)                                                                         
                                                                                                  
 expanded_

                                                                 rm[0][0]']                       
                                                                                                  
 expanded_conv_6/expand (Conv2D  (None, 14, 14, 120)  4800       ['expanded_conv_5/Add[0][0]']    
 )                                                                                                
                                                                                                  
 expanded_conv_6/expand/BatchNo  (None, 14, 14, 120)  480        ['expanded_conv_6/expand[0][0]'] 
 rm (BatchNormalization)                                                                          
                                                                                                  
 tf.__operators__.add_11 (TFOpL  (None, 14, 14, 120)  0          ['expanded_conv_6/expand/BatchNor
 ambda)                                                          m[0][0]']                        
          

                                                                 m[0][0]',                        
                                                                  'tf.math.multiply_14[0][0]']    
                                                                                                  
 expanded_conv_7/depthwise (Dep  (None, 14, 14, 144)  3600       ['multiply_9[0][0]']             
 thwiseConv2D)                                                                                    
                                                                                                  
 expanded_conv_7/depthwise/Batc  (None, 14, 14, 144)  576        ['expanded_conv_7/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 tf.__operators__.add_15 (TFOpL  (None, 14, 14, 144)  0          ['expanded_conv_7/depthwise/Batch
 ambda)   

 tf.__operators__.add_18 (TFOpL  (None, 7, 7, 288)   0           ['expanded_conv_8/depthwise/Batch
 ambda)                                                          Norm[0][0]']                     
                                                                                                  
 re_lu_23 (ReLU)                (None, 7, 7, 288)    0           ['tf.__operators__.add_18[0][0]']
                                                                                                  
 tf.math.multiply_18 (TFOpLambd  (None, 7, 7, 288)   0           ['re_lu_23[0][0]']               
 a)                                                                                               
                                                                                                  
 multiply_12 (Multiply)         (None, 7, 7, 288)    0           ['expanded_conv_8/depthwise/Batch
                                                                 Norm[0][0]',                     
          

 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 144)   0           ['expanded_conv_9/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 576)   83520       ['expanded_conv_9/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__operators__.add_22 (TFOpL  (None, 1, 1, 576)   0           ['expanded_conv_9/squeeze_excite/
 ambda)                                                          Conv_1[0][0]']                   
          

                                                                                                  
 tf.math.multiply_25 (TFOpLambd  (None, 1, 1, 576)   0           ['re_lu_30[0][0]']               
 a)                                                                                               
                                                                                                  
 expanded_conv_10/squeeze_excit  (None, 7, 7, 576)   0           ['multiply_16[0][0]',            
 e/Mul (Multiply)                                                 'tf.math.multiply_25[0][0]']    
                                                                                                  
 expanded_conv_10/project (Conv  (None, 7, 7, 96)    55296       ['expanded_conv_10/squeeze_excite
 2D)                                                             /Mul[0][0]']                     
                                                                                                  
 expanded_

In [9]:
inputs = keras.layers.Input((224,224,3))
#None means this dimension is variable 

feature_batch = base_model(inputs)
print(feature_batch.shape)

(None, 7, 7, 576)


In [19]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(None, 576)


In [20]:
prediction_layer = keras.layers.Dense(7)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(None, 7)


In [21]:
model = tf.keras.Sequential([ #functional api 
  base_model,
  global_average_layer,
  prediction_layer
])

In [22]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### model with (trainable = False) 

In [23]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3small (Functiona  (None, 7, 7, 576)        939120    
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 576)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 7)                 4039      
                                                                 
Total params: 943,159
Trainable params: 4,039
Non-trainable params: 939,120
_________________________________________________________________


### conv10 unfreezing test

In [56]:
#for i in base_model.layers[-29:]:
#    print(i.name, i.trainable)
base_model.trainable = False 
for i in base_model.layers[-29:]:
    i.trainable = True 
    
for i in base_model.layers[-30:]:
    print(i.name, i.trainable)

expanded_conv_9/Add False
expanded_conv_10/expand True
expanded_conv_10/expand/BatchNorm True
tf.__operators__.add_104 True
re_lu_124 True
tf.math.multiply_104 True
multiply_69 True
expanded_conv_10/depthwise True
expanded_conv_10/depthwise/BatchNorm True
tf.__operators__.add_105 True
re_lu_125 True
tf.math.multiply_105 True
multiply_70 True
expanded_conv_10/squeeze_excite/AvgPool True
expanded_conv_10/squeeze_excite/Conv True
expanded_conv_10/squeeze_excite/Relu True
expanded_conv_10/squeeze_excite/Conv_1 True
tf.__operators__.add_106 True
re_lu_126 True
tf.math.multiply_106 True
expanded_conv_10/squeeze_excite/Mul True
expanded_conv_10/project True
expanded_conv_10/project/BatchNorm True
expanded_conv_10/Add True
Conv_1 True
Conv_1/BatchNorm True
tf.__operators__.add_107 True
re_lu_127 True
tf.math.multiply_107 True
multiply_71 True


In [32]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Adam, RMSprop, Nadam, NAG 

### Adam

In [33]:
history = model.fit(training_ds, validation_data = validation_ds, epochs = 5, use_multiprocessing = True, workers =16)

Epoch 1/5
10927/10927 [==============================] - 364s 33ms/step - loss: 1.4529 - accuracy: 0.4173 - val_loss: 1.3162 - val_accuracy: 0.4829
Epoch 2/5
10927/10927 [==============================] - 361s 33ms/step - loss: 1.3004 - accuracy: 0.4863 - val_loss: 1.2568 - val_accuracy: 0.5062
Epoch 3/5
10927/10927 [==============================] - 364s 33ms/step - loss: 1.2587 - accuracy: 0.5038 - val_loss: 1.2287 - val_accuracy: 0.5159
Epoch 4/5
10927/10927 [==============================] - 368s 34ms/step - loss: 1.2378 - accuracy: 0.5119 - val_loss: 1.2128 - val_accuracy: 0.5215
Epoch 5/5
10927/10927 [==============================] - 362s 33ms/step - loss: 1.2236 - accuracy: 0.5180 - val_loss: 1.2026 - val_accuracy: 0.5260


### Nadam

In [48]:
optimizers = ['Nadam', 'RMSprop', 'Adamax', 'SGD', 'Adam']
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print('Nadam')
Nadam = model.fit(training_ds, validation_data = validation_ds, epochs = 3, use_multiprocessing = True, workers =16)

Nadam
Epoch 1/3
10927/10927 [==============================] - 363s 33ms/step - loss: 1.4586 - accuracy: 0.4159 - val_loss: 1.3169 - val_accuracy: 0.4826
Epoch 2/3
10927/10927 [==============================] - 364s 33ms/step - loss: 1.3000 - accuracy: 0.4885 - val_loss: 1.2556 - val_accuracy: 0.5056
Epoch 3/3
10927/10927 [==============================] - 363s 33ms/step - loss: 1.2579 - accuracy: 0.5040 - val_loss: 1.2275 - val_accuracy: 0.5175


### RMSprop

In [58]:
optimizers = ['Nadam', 'RMSprop', 'Adamax', 'SGD', 'Adam']
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print('RMSprop')
RMSprop = model.fit(training_ds, validation_data = validation_ds, epochs = 3, use_multiprocessing = True, workers =16)

RMSprop
Epoch 1/3
10927/10927 [==============================] - 365s 33ms/step - loss: 1.4565 - accuracy: 0.4169 - val_loss: 1.3197 - val_accuracy: 0.4793
Epoch 2/3
10927/10927 [==============================] - 360s 33ms/step - loss: 1.3020 - accuracy: 0.4869 - val_loss: 1.2570 - val_accuracy: 0.5056
Epoch 3/3
10927/10927 [==============================] - 359s 33ms/step - loss: 1.2603 - accuracy: 0.5030 - val_loss: 1.2298 - val_accuracy: 0.5147


### try with weights = none 

In [10]:
base_model_RI = tf.keras.applications.MobileNetV3Small(input_shape = (224,224,3), include_top = False, weights = None)
base_model_RI.trainable = True
base_model_RI.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                   

 expanded_conv_1/depthwise/Batc  (None, 28, 28, 72)  288         ['expanded_conv_1/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 re_lu_36 (ReLU)                (None, 28, 28, 72)   0           ['expanded_conv_1/depthwise/Batch
                                                                 Norm[0][0]']                     
                                                                                                  
 expanded_conv_1/project (Conv2  (None, 28, 28, 24)  1728        ['re_lu_36[0][0]']               
 D)                                                                                               
                                                                                                  
 expanded_conv_1/project/BatchN  (None, 28, 28, 24)  96          ['expanded_conv_1/project[0][0]']
 orm (Batc

                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    2328        ['expanded_conv_3/squeeze_excite/
 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    0           ['expanded_conv_3/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 96)    2400        ['expanded_conv_3/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__oper

                                                                                                  
 expanded_conv_4/squeeze_excite  (None, 14, 14, 240)  0          ['multiply_22[0][0]',            
 /Mul (Multiply)                                                  'tf.math.multiply_34[0][0]']    
                                                                                                  
 expanded_conv_4/project (Conv2  (None, 14, 14, 40)  9600        ['expanded_conv_4/squeeze_excite/
 D)                                                              Mul[0][0]']                      
                                                                                                  
 expanded_conv_4/project/BatchN  (None, 14, 14, 40)  160         ['expanded_conv_4/project[0][0]']
 orm (BatchNormalization)                                                                         
                                                                                                  
 expanded_

                                                                 rm[0][0]']                       
                                                                                                  
 expanded_conv_6/expand (Conv2D  (None, 14, 14, 120)  4800       ['expanded_conv_5/Add[0][0]']    
 )                                                                                                
                                                                                                  
 expanded_conv_6/expand/BatchNo  (None, 14, 14, 120)  480        ['expanded_conv_6/expand[0][0]'] 
 rm (BatchNormalization)                                                                          
                                                                                                  
 tf.__operators__.add_38 (TFOpL  (None, 14, 14, 120)  0          ['expanded_conv_6/expand/BatchNor
 ambda)                                                          m[0][0]']                        
          

                                                                 m[0][0]',                        
                                                                  'tf.math.multiply_41[0][0]']    
                                                                                                  
 expanded_conv_7/depthwise (Dep  (None, 14, 14, 144)  3600       ['multiply_27[0][0]']            
 thwiseConv2D)                                                                                    
                                                                                                  
 expanded_conv_7/depthwise/Batc  (None, 14, 14, 144)  576        ['expanded_conv_7/depthwise[0][0]
 hNorm (BatchNormalization)                                      ']                               
                                                                                                  
 tf.__operators__.add_42 (TFOpL  (None, 14, 14, 144)  0          ['expanded_conv_7/depthwise/Batch
 ambda)   

 tf.__operators__.add_45 (TFOpL  (None, 7, 7, 288)   0           ['expanded_conv_8/depthwise/Batch
 ambda)                                                          Norm[0][0]']                     
                                                                                                  
 re_lu_55 (ReLU)                (None, 7, 7, 288)    0           ['tf.__operators__.add_45[0][0]']
                                                                                                  
 tf.math.multiply_45 (TFOpLambd  (None, 7, 7, 288)   0           ['re_lu_55[0][0]']               
 a)                                                                                               
                                                                                                  
 multiply_30 (Multiply)         (None, 7, 7, 288)    0           ['expanded_conv_8/depthwise/Batch
                                                                 Norm[0][0]',                     
          

 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 144)   0           ['expanded_conv_9/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
                                                                                                  
 expanded_conv_9/squeeze_excite  (None, 1, 1, 576)   83520       ['expanded_conv_9/squeeze_excite/
 /Conv_1 (Conv2D)                                                Relu[0][0]']                     
                                                                                                  
 tf.__operators__.add_49 (TFOpL  (None, 1, 1, 576)   0           ['expanded_conv_9/squeeze_excite/
 ambda)                                                          Conv_1[0][0]']                   
          

                                                                                                  
 tf.math.multiply_52 (TFOpLambd  (None, 1, 1, 576)   0           ['re_lu_62[0][0]']               
 a)                                                                                               
                                                                                                  
 expanded_conv_10/squeeze_excit  (None, 7, 7, 576)   0           ['multiply_34[0][0]',            
 e/Mul (Multiply)                                                 'tf.math.multiply_52[0][0]']    
                                                                                                  
 expanded_conv_10/project (Conv  (None, 7, 7, 96)    55296       ['expanded_conv_10/squeeze_excite
 2D)                                                             /Mul[0][0]']                     
                                                                                                  
 expanded_

In [11]:
inputs = keras.layers.Input((224,224,3))
#None means this dimension is variable 

feature_batch = base_model_RI(inputs)
print(feature_batch.shape)

(None, 7, 7, 576)


In [12]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(None, 576)


In [13]:
prediction_layer = keras.layers.Dense(7)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(None, 7)


In [14]:
model_RI = tf.keras.Sequential([ #functional api 
  base_model_RI,
  global_average_layer,
  prediction_layer
])

In [15]:
base_learning_rate = 0.0001
model_RI.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
wo_imagenet = model_RI.fit(training_ds, validation_data = validation_ds, epochs = 3, use_multiprocessing = True, workers =16)

Epoch 1/3


2022-08-17 13:24:24.831675: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 957s 75ms/step - loss: 1.1474 - accuracy: 0.5616 - val_loss: 0.9522 - val_accuracy: 0.6505
Epoch 2/3
12495/12495 [==============================] - 954s 76ms/step - loss: 0.8752 - accuracy: 0.6733 - val_loss: 0.8700 - val_accuracy: 0.6816
Epoch 3/3
12495/12495 [==============================] - 948s 76ms/step - loss: 0.7778 - accuracy: 0.7107 - val_loss: 0.8160 - val_accuracy: 0.7038


In [42]:
AfNet_val_ds = tf.keras.utils.image_dataset_from_directory(
    AffectNet_val+"seven/",
    labels = "inferred",
    image_size = (224,224),
    )

Found 3500 files belonging to 7 classes.


In [40]:
model_RI.evaluate(AfNet_val_ds, use_multiprocessing=True, workers =16)


3500/3500 [==============================] - 35s 10ms/step - loss: 2.4455 - accuracy: 0.2729


[2.445486307144165, 0.272857129573822]

In [17]:
AfNet_train_ds = tf.keras.utils.image_dataset_from_directory(
    AffectNet_train+"seven/",
    labels = "inferred",
    image_size = (224,224),
    )

Found 283901 files belonging to 7 classes.


In [18]:
model_RI.evaluate(AfNet_train_ds, use_multiprocessing=True, workers =16)


8872/8872 [==============================] - 1103s 124ms/step - loss: 3.1004 - accuracy: 0.1275


[3.1004478931427, 0.1275479793548584]